In [1]:
import pandas as pd
import numpy as np
# TensorFlow と tf.keras のインポート
import tensorflow as tf
from tensorflow import keras

# ヘルパーライブラリのインポート
import matplotlib.pyplot as plt

print(tf.__version__)

2.4.1


In [2]:
# csvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('train.csv', header=None, delimiter=',', low_memory=False, nrows=700)
test = pd.read_csv('train.csv', header=None, delimiter=',', low_memory=False, skiprows=700, nrows=300)

In [3]:
#冒頭を表示
train.head()

,0,1,2,3,4,5,6
0,0.628588,0.609606,-0.155139,0.388005,0.492012,0.383191,C4
1,0.467862,0.813979,0.139157,-0.744689,0.090100,0.380830,C2
2,-0.664520,0.505091,-0.391892,-0.275558,-0.029550,-0.335643,C1
3,0.834265,-0.148955,-0.455550,-0.204187,0.916190,-0.124268,C5
4,0.082797,-0.181984,-0.354305,0.415666,0.930169,-0.015068,C4


In [4]:
#冒頭を表示
test.head()

,0,1,2,3,4,5,6
0,-0.017255,-0.862866,0.967365,-0.407979,0.154430,0.014889,C5
1,-0.789765,-0.838736,0.994123,0.231924,-0.093740,0.662405,C4
2,-0.319167,-0.027778,0.830249,-0.307585,0.751789,0.008866,C4
3,-0.591453,0.350734,0.176879,0.403390,0.842161,-0.207443,C3
4,-0.121794,-0.144442,0.567281,-0.578433,0.764733,0.017592,C4


In [5]:
#形を確認
train.shape, test.shape

((700, 7), (300, 7))

In [6]:
#データのタイプを確認
train.dtypes

0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6     object
dtype: object

In [7]:
#データのタイプを確認
test.dtypes

0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6     object
dtype: object

In [8]:
#訓練データの詳細を確認
train.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
0,700,NaN,NaN,NaN,0.0286345,0.574669,-0.997604,-0.43946,0.0261777,0.523148,0.999615
1,700,NaN,NaN,NaN,-0.000550779,0.575807,-0.994549,-0.503909,0.0297762,0.4941,0.99976
2,700,NaN,NaN,NaN,4.14671e-05,0.575869,-0.998664,-0.507913,0.00184063,0.510043,0.998575
3,700,NaN,NaN,NaN,0.0311201,0.569434,-0.999538,-0.454879,0.0418686,0.530797,0.999756
4,700,NaN,NaN,NaN,-0.0197487,0.585133,-0.99481,-0.509058,-0.0232711,0.497322,0.99695
5,700,NaN,NaN,NaN,0.00437691,0.333845,-0.981412,-0.158998,0.00642199,0.193134,0.951476
6,700,5,C4,152,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#訓練データのユニーク値を確認
train.apply(lambda x: x.nunique())

0    700
1    700
2    700
3    700
4    700
5    700
6      5
dtype: int64

In [10]:
#trainのtargetをカテゴリーに変換
train[6] = train[6].astype('category')
test[6] = test[6].astype('category')

train[6].dtype, test[6].dtype

(CategoricalDtype(categories=['C1', 'C2', 'C3', 'C4', 'C5'], ordered=False),
 CategoricalDtype(categories=['C1', 'C2', 'C3', 'C4', 'C5'], ordered=False))

In [11]:
# ラベルエンコーディング（LabelEncoder）
from sklearn.preprocessing import LabelEncoder

#学習データ
le = LabelEncoder()
encoded = le.fit_transform(train[6].values)
decoded = le.inverse_transform(encoded)
train[6] = encoded
#テストデータ
le = LabelEncoder()
encoded = le.fit_transform(test[6].values)
decoded = le.inverse_transform(encoded)
test[6] = encoded

#冒頭を表示
train[6].head(), test[6].head()

(0    3
 1    1
 2    0
 3    4
 4    3
 Name: 6, dtype: int64,
 0    4
 1    3
 2    3
 3    2
 4    3
 Name: 6, dtype: int64)

In [12]:
#冒頭を表示
train.head()

,0,1,2,3,4,5,6
0,0.628588,0.609606,-0.155139,0.388005,0.492012,0.383191,3
1,0.467862,0.813979,0.139157,-0.744689,0.090100,0.380830,1
2,-0.664520,0.505091,-0.391892,-0.275558,-0.029550,-0.335643,0
3,0.834265,-0.148955,-0.455550,-0.204187,0.916190,-0.124268,4
4,0.082797,-0.181984,-0.354305,0.415666,0.930169,-0.015068,3


In [13]:
#訓練データとテストデータを分割する
x_train, y_train = train.drop([6], axis=1), train[6]
x_test, y_test = test.drop([6], axis=1), test[6]

In [14]:
#冒頭を表示
x_train.head()

,0,1,2,3,4,5
0,0.628588,0.609606,-0.155139,0.388005,0.492012,0.383191
1,0.467862,0.813979,0.139157,-0.744689,0.090100,0.380830
2,-0.664520,0.505091,-0.391892,-0.275558,-0.029550,-0.335643
3,0.834265,-0.148955,-0.455550,-0.204187,0.916190,-0.124268
4,0.082797,-0.181984,-0.354305,0.415666,0.930169,-0.015068


In [15]:
#冒頭を表示
y_train.head()

0    3
1    1
2    0
3    4
4    3
Name: 6, dtype: int64

In [16]:
#モデルを構築
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.0),
    keras.layers.Dense(5, activation='softmax')
])

In [17]:
#モデルをコンパイルする
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
#学習データを適用する
model.fit(x_train, y_train, epochs=475, batch_size=32)

Epoch 1/475
22/22 [==============================] - 1s 974us/step - loss: 1.5954 - accuracy: 0.1780
Epoch 2/475
22/22 [==============================] - 0s 974us/step - loss: 1.5688 - accuracy: 0.2281
Epoch 3/475
22/22 [==============================] - 0s 1ms/step - loss: 1.5338 - accuracy: 0.2414
Epoch 4/475
22/22 [==============================] - 0s 967us/step - loss: 1.4893 - accuracy: 0.3762
Epoch 5/475
22/22 [==============================] - 0s 999us/step - loss: 1.4439 - accuracy: 0.4270
Epoch 6/475
22/22 [==============================] - 0s 883us/step - loss: 1.3663 - accuracy: 0.4370
Epoch 7/475
22/22 [==============================] - 0s 929us/step - loss: 1.2979 - accuracy: 0.4354
Epoch 8/475
22/22 [==============================] - 0s 920us/step - loss: 1.2355 - accuracy: 0.4736
Epoch 9/475
22/22 [==============================] - 0s 899us/step - loss: 1.1438 - accuracy: 0.5116
Epoch 10/475
22/22 [==============================] - 0s 885us/step - loss: 1.0936 - accuracy

22/22 [==============================] - 0s 893us/step - loss: 0.1939 - accuracy: 0.9305
Epoch 82/475
22/22 [==============================] - 0s 878us/step - loss: 0.2406 - accuracy: 0.8971
Epoch 83/475
22/22 [==============================] - 0s 895us/step - loss: 0.2715 - accuracy: 0.8910
Epoch 84/475
22/22 [==============================] - 0s 839us/step - loss: 0.2366 - accuracy: 0.9009
Epoch 85/475
22/22 [==============================] - 0s 836us/step - loss: 0.2210 - accuracy: 0.9108
Epoch 86/475
22/22 [==============================] - 0s 869us/step - loss: 0.2068 - accuracy: 0.9264
Epoch 87/475
22/22 [==============================] - 0s 843us/step - loss: 0.2606 - accuracy: 0.9050
Epoch 88/475
22/22 [==============================] - 0s 864us/step - loss: 0.2119 - accuracy: 0.9261
Epoch 89/475
22/22 [==============================] - 0s 839us/step - loss: 0.2037 - accuracy: 0.9271
Epoch 90/475
22/22 [==============================] - 0s 842us/step - loss: 0.2060 - accuracy: 

22/22 [==============================] - 0s 810us/step - loss: 0.1737 - accuracy: 0.9304
Epoch 161/475
22/22 [==============================] - 0s 947us/step - loss: 0.1756 - accuracy: 0.9297
Epoch 162/475
22/22 [==============================] - 0s 837us/step - loss: 0.1659 - accuracy: 0.9365
Epoch 163/475
22/22 [==============================] - 0s 837us/step - loss: 0.1565 - accuracy: 0.9401
Epoch 164/475
22/22 [==============================] - 0s 823us/step - loss: 0.1750 - accuracy: 0.9258
Epoch 165/475
22/22 [==============================] - 0s 815us/step - loss: 0.1446 - accuracy: 0.9428
Epoch 166/475
22/22 [==============================] - 0s 829us/step - loss: 0.1692 - accuracy: 0.9300
Epoch 167/475
22/22 [==============================] - 0s 858us/step - loss: 0.1521 - accuracy: 0.9462
Epoch 168/475
22/22 [==============================] - 0s 862us/step - loss: 0.1532 - accuracy: 0.9365
Epoch 169/475
22/22 [==============================] - 0s 930us/step - loss: 0.1681 - a

22/22 [==============================] - 0s 865us/step - loss: 0.1195 - accuracy: 0.9433
Epoch 240/475
22/22 [==============================] - 0s 916us/step - loss: 0.1319 - accuracy: 0.9446
Epoch 241/475
22/22 [==============================] - 0s 805us/step - loss: 0.1157 - accuracy: 0.9584
Epoch 242/475
22/22 [==============================] - 0s 772us/step - loss: 0.0794 - accuracy: 0.9709
Epoch 243/475
22/22 [==============================] - 0s 821us/step - loss: 0.1176 - accuracy: 0.9477
Epoch 244/475
22/22 [==============================] - 0s 983us/step - loss: 0.0971 - accuracy: 0.9752
Epoch 245/475
22/22 [==============================] - 0s 895us/step - loss: 0.1177 - accuracy: 0.9558
Epoch 246/475
22/22 [==============================] - 0s 898us/step - loss: 0.1138 - accuracy: 0.9476
Epoch 247/475
22/22 [==============================] - 0s 904us/step - loss: 0.1312 - accuracy: 0.9422
Epoch 248/475
22/22 [==============================] - 0s 842us/step - loss: 0.1446 - a

22/22 [==============================] - 0s 813us/step - loss: 0.0821 - accuracy: 0.9723
Epoch 319/475
22/22 [==============================] - 0s 835us/step - loss: 0.0678 - accuracy: 0.9783
Epoch 320/475
22/22 [==============================] - 0s 810us/step - loss: 0.0909 - accuracy: 0.9667
Epoch 321/475
22/22 [==============================] - 0s 833us/step - loss: 0.0981 - accuracy: 0.9610
Epoch 322/475
22/22 [==============================] - 0s 835us/step - loss: 0.1072 - accuracy: 0.9523
Epoch 323/475
22/22 [==============================] - 0s 888us/step - loss: 0.0917 - accuracy: 0.9694
Epoch 324/475
22/22 [==============================] - 0s 884us/step - loss: 0.0965 - accuracy: 0.9572
Epoch 325/475
22/22 [==============================] - 0s 859us/step - loss: 0.1032 - accuracy: 0.9710
Epoch 326/475
22/22 [==============================] - 0s 848us/step - loss: 0.0865 - accuracy: 0.9724
Epoch 327/475
22/22 [==============================] - 0s 858us/step - loss: 0.0866 - a

22/22 [==============================] - 0s 873us/step - loss: 0.0664 - accuracy: 0.9743
Epoch 398/475
22/22 [==============================] - 0s 882us/step - loss: 0.0903 - accuracy: 0.9632
Epoch 399/475
22/22 [==============================] - 0s 801us/step - loss: 0.0648 - accuracy: 0.9776
Epoch 400/475
22/22 [==============================] - 0s 931us/step - loss: 0.0751 - accuracy: 0.9696
Epoch 401/475
22/22 [==============================] - 0s 832us/step - loss: 0.0686 - accuracy: 0.9749
Epoch 402/475
22/22 [==============================] - 0s 791us/step - loss: 0.0998 - accuracy: 0.9712
Epoch 403/475
22/22 [==============================] - 0s 778us/step - loss: 0.0617 - accuracy: 0.9768
Epoch 404/475
22/22 [==============================] - 0s 786us/step - loss: 0.0684 - accuracy: 0.9738
Epoch 405/475
22/22 [==============================] - 0s 831us/step - loss: 0.0491 - accuracy: 0.9907
Epoch 406/475
22/22 [==============================] - 0s 827us/step - loss: 0.0633 - a

In [19]:
#テストデータに適用
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

10/10 - 0s - loss: 0.0662 - accuracy: 0.9800

Test accuracy: 0.9800000190734863
